In [3]:
%cd D:\Project\Toolkit_for_Preprocessing_MXH\ViHOS_tokenize

from IPython.display import clear_output
clear_output()

In [5]:
import tensorflow as tf
import torch

tf.keras.backend.clear_session()
# clear gpu memory using torch
torch.cuda.empty_cache()
# clear output
clear_output()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [6]:
train_path = (r"Data\BIO_data\train_BIO.csv")
dev_path = (r"Data\BIO_data\dev_BIO.csv")
test_path = (r"Data\BIO_data\test_BIO.csv")
test_index = 50 # default None value
batch_size = 64
max_len = 64
shuffle = False

In [7]:
from transformers import (
    XLMRobertaModel,
    AutoTokenizer
)

input_model = XLMRobertaModel.from_pretrained("vinai/phobert-base")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
input_model.resize_token_embeddings(len(tokenizer))
clear_output()

In [8]:
if test_index != None and test_index > 3:
    # Load the data
    train_path, dev_path, test_path = split_path(test_path, test_index, train_path, dev_path, test_path)
elif test_index != None: 
    print("Test index out of range. Please provide a valid interger index greater than 3.")

Training set: 363 samples
Test set: 102 samples
Development set: 125 samples


In [9]:
from Code.Dataset import split_path, create_dataloader

train_dataloader = create_dataloader(train_path, batch_size=batch_size, tokenizer = tokenizer, max_len=max_len)
dev_dataloader = create_dataloader(dev_path, batch_size=batch_size, tokenizer = tokenizer, max_len=max_len, shuffle=shuffle)
test_dataloader = create_dataloader(test_path, batch_size=batch_size, tokenizer = tokenizer, max_len=max_len)

Columns: Index(['Unnamed: 0', 'index', 'Word', 'Tag', 'sentence_id'], dtype='object')
Columns: Index(['Unnamed: 0', 'index', 'Word', 'Tag', 'sentence_id'], dtype='object')
Columns: Index(['Unnamed: 0', 'index', 'Word', 'Tag', 'sentence_id'], dtype='object')


In [16]:
from Code.Model import setup_model, MultiTaskModel, train, test

# Set up the model and training components
model, criterion_span, optimizer_spans, device, num_epochs = setup_model(
    input_model=input_model,
    model_class=MultiTaskModel,
    lr=5e-6,
    weight_decay=1e-5,
    num_epochs=2
)

# Now you can call your train and test functions with the returned objects
train(
    model=model,
    train_dataloader=train_dataloader,
    dev_dataloader=dev_dataloader,
    criterion_span=criterion_span,
    optimizer_spans=optimizer_spans,
    device=device,
    num_epochs=num_epochs
)

# Testing the model after training
span_preds, span_targets = test(
    model=model,
    test_dataloader=test_dataloader,
    device=device
)


Epoch: 1


Training Loss: 0.5727
Validation Loss: 0.4313
Span Macro F1-Score: 0.4986
Epoch: 2


Training Loss: 0.4113
Validation Loss: 0.3138
Span Macro F1-Score: 0.4987


Testing: 100%|██████████| 2/2 [00:08<00:00,  4.35s/it]

Span F1 Score: 0.4994


In [17]:
import json
test_results = {
    "predictions": span_preds.tolist(),
    "targets": span_targets.tolist()
}
with open('result.json', 'w') as f:
    json.dump(test_results, f, indent=4)
    # print(f"Test results saved to {args.output_json}")

    # Save the trained model
    model_save_path ="trained_model.pth"
    torch.save(model.state_dict(), model_save_path)
    print(f"Model saved to {model_save_path}")

Model saved to trained_model.pth


In [18]:
!python main.py --train_path "Data/BIO_data/train_BIO.csv" \
                     --dev_path "Data/BIO_data/dev_BIO.csv" \
                     --test_path "Data/BIO_data/test_BIO.csv" \
                     --test_index 50 \
                     --batch_size 64 \
                     --max_len 64 \
                     --lr 5e-6 \
                     --num_epochs 2 \
                     --output_json "test_results.json" \
                     --output_dir "output"


Traceback (most recent call last):
  File "D:\Project\Toolkit_for_Preprocessing_MXH\ViHOS_tokenize\Code\main.py", line 7, in <module>
    from Code.Dataset import split_path, create_dataloader
ModuleNotFoundError: No module named 'Code'
